In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
import torch
import numpy as np
import random
from copy import deepcopy
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0) 
#torch.backends.cudnn.deterministic = True

In [ ]:
from torchvision import datasets
MNIST_train = datasets.MNIST(root='./', train=True,
                                   download=True)
MNIST_test = datasets.MNIST(root='./', train=False,
                                  download=False)

In [ ]:

X_train = MNIST_train.data
y_train = MNIST_train.targets
X_test = MNIST_test.data
y_test = MNIST_test.targets

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_indices_01 = []
train_indices_23 = []
train_indices_45 = []
train_indices_67 = []
train_indices_89 = []
test_indices_01 = []
test_indices_23 = []
test_indices_45 = []
test_indices_67 = []
test_indices_89 = []
for i in range(len(y_train)):
  if y_train[i] == 0 or y_train[i] == 1:
    train_indices_01.append(i)
  elif y_train[i] == 2 or y_train[i] == 3:
    train_indices_23.append(i)
  elif y_train[i] == 4 or y_train[i] == 5:
    train_indices_45.append(i)
  elif y_train[i] == 6 or y_train[i] == 7:
    train_indices_67.append(i)
  elif y_train[i] == 8 or y_train[i] == 9:
    train_indices_89.append(i)
  


for i in range(len(y_test)):
  if y_test[i] == 0 or y_test[i] == 1:
    test_indices_01.append(i)
  elif y_test[i] == 2 or y_test[i]== 3:
    test_indices_23.append(i)
  elif y_test[i] == 4 or y_test[i] == 5:
    test_indices_45.append(i)
  elif y_test[i] == 6 or y_test[i] == 7:
    test_indices_67.append(i)
  elif y_test[i] == 8 or y_test[i] == 9:
    test_indices_89.append(i)



X_train_01 = X_train[train_indices_01]
y_train_01 = y_train[train_indices_01]
X_test_01 = X_test[test_indices_01]
y_test_01 = y_test[test_indices_01]

X_train_23 = X_train[train_indices_23]
y_train_23 = y_train[train_indices_23]
X_test_23 = X_test[test_indices_23]
y_test_23 = y_test[test_indices_23]

X_train_45 = X_train[train_indices_45]
y_train_45 = y_train[train_indices_45]
X_test_45 = X_test[test_indices_45]
y_test_45 = y_test[test_indices_45]

X_train_67 = X_train[train_indices_67]
y_train_67 = y_train[train_indices_67]
X_test_67 = X_test[test_indices_67]
y_test_67 = y_test[test_indices_67]

X_train_89 = X_train[train_indices_89]
y_train_89 = y_train[train_indices_89]
X_test_89 = X_test[test_indices_89]
y_test_89 = y_test[test_indices_89]


X_test_01 = X_test_01.to(device)
#print(type(y_train_01))
#print(y_train_01[0])


X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()

X_train_01 = X_train_01.unsqueeze(1).float()
X_test_01 = X_test_01.unsqueeze(1).float()

X_train_23 = X_train_23.unsqueeze(1).float()
X_test_23 = X_test_23.unsqueeze(1).float()

X_train_45 = X_train_45.unsqueeze(1).float()
X_test_45 = X_test_45.unsqueeze(1).float()

X_train_67 = X_train_67.unsqueeze(1).float()
X_test_67 = X_test_67.unsqueeze(1).float()

X_train_89 = X_train_89.unsqueeze(1).float()
X_test_89 = X_test_89.unsqueeze(1).float()

In [ ]:
class YannLecun(torch.nn.Module):
  def __init__(self,activation = 'leaky relu', pooling = 'max', conv_size = 3, use_batch_norm = False, use_dropout = False):
    super(YannLecun, self).__init__()
    self.conv_size = conv_size
    self.use_batch_norm = use_batch_norm
    self.use_dropout = use_dropout

    if activation == 'relu':
      activation_function = torch.nn.ReLU()
    elif activation == 'tanh':
      activation_function = torch.nn.Tanh()
    elif activation == 'leaky relu':
        activation_function = torch.nn.LeakyReLU()
    else:
      raise NotImplementedError   

    if pooling == 'max':
      pooling_layer = torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    elif pooling == 'avg':
      pooling_layer = torch.nn.AvgPool2d(kernel_size = 2, stride = 2)
    else:
      raise NotImplementedError   
    
    if conv_size == 3:
      self.conv1_1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, padding= 1)
      self.conv1_2 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding= 1)
    elif conv_size == 5:
      self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding= 2)
    else:
      raise NotImplementedError
    
    self.act1 = activation_function
    self.bn1 = torch.nn.BatchNorm2d(num_features = 6)
    self.pool1 = pooling_layer
    self.dp1 = torch.nn.Dropout(p = 0.25) 

    if conv_size == 3:
      self.conv2_1 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, padding= 0)
      self.conv2_2 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding= 0)
    elif conv_size == 5:
      self.conv2 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding= 0)
    else:
      raise NotImplementedError

    self.act2 = activation_function
    self.bn2 = torch.nn.BatchNorm2d(num_features = 16)
    self.pool2 = pooling_layer
    self.dp2 = torch.nn.Dropout(p = 0.25) 

    self.fc1 = torch.nn.Linear(5*5*16, 120)
    self.act3 = torch.nn.Tanh()
    self.dp3 = torch.nn.Dropout(p = 0.5) 

    self.fc2 = torch.nn.Linear(120,84)
    self.act4 = torch.nn.Tanh()
    self.dp4 = torch.nn.Dropout(p = 0.5) 

    self.fc3 = torch.nn.Linear(84,10)
    self.act5 = torch.nn.Softmax()


  def forward(self,x):
    if self.conv_size == 3:
      x = self.conv1_1(x)
      x = self.conv1_2(x)
    elif self.conv_size == 5:
      x = self.conv1(x)

    x = self.act1(x)
    if self.use_batch_norm:
      x = self.bn1(x)
    x = self.pool1(x)
    if self.use_dropout:
      x = self.dp1(x)


    if self.conv_size == 3:
      x = self.conv2_1(x)
      x = self.conv2_2(x)
    elif self.conv_size == 5:
      x = self.conv2(x)
    
    x = self.act2(x)
    if self.use_batch_norm:
      x = self.bn2(x)
    x = self.pool2(x)
    if self.use_dropout:
      x = self.dp2(x)

    x = x.view(x.size(0),x.size(1)*x.size(2)*x.size(3))

    x = self.fc1(x)
    x = self.act3(x)
    if self.use_dropout:
      x = self.dp3(x)
    x = self.fc2(x)
    x = self.act4(x)
    if self.use_dropout:
      x = self.dp4(x)
    x = self.fc3(x)
    x = self.act5(x)
    
    x = x.to(device)

    return x

yann_lecun = YannLecun(use_batch_norm = True, use_dropout =True)
yann_lecun = yann_lecun.to(device)
loss = torch.nn.CrossEntropyLoss()
my_optimizer = torch.optim.Adam(yann_lecun.parameters(), lr=4.0e-4)

In [ ]:
def variable(t: torch.Tensor, use_cuda=True, **kwargs): #переменная тупо для того, чтобы можно было дифференцировать по ним.
    if torch.cuda.is_available() and use_cuda:          #в нашем случае в самом тензоре есть requires_grad если True то переменная
        t = t.cuda()
    return Variable(t, **kwargs)


def test_accuracy(x_test, y_test):
    x_test = x_test.to(device)
    test_preds = yann_lecun.forward(x_test)
    test_preds = test_preds.to(device)
    y_test = y_test.to(device)
    accuracy = (test_preds.argmax(dim = 1) == y_test).float().mean().data.cpu()
    return accuracy

class EWC(object):
    def __init__(self, old_x, old_y = None, model = yann_lecun):

        self.model = model
        self.old_x = old_x # по сути иксы
        self.old_y = old_y
        self.params = {n: p for n, p in self.model.named_parameters() if p.requires_grad} #собирает в словарь имя:параметры
        self._old_params = {}
        self._precision_matrices = self._diag_fisher() 

        for n, p in deepcopy(self.params).items():
            self._old_params[n] = variable(p.data)


    def _diag_fisher(self):
        precision_matrices = {}
        for n, p in deepcopy(self.params).items(): #копируем параметры через дип копи чтоб не поменять их
            p.data.zero_() #все параметры скопированные зануляем
            precision_matrices[n] = variable(p.data) #выставляем requires_grad = True 

        self.model.eval() #переводим в состояние evaluation
        #for input in self.dataset:
        self.model.zero_grad() #как обычно чтоб не накапливался градиент зануляем вначале
        self.old_x = variable(self.old_x) #input requires grad = True
        output = self.model(self.old_x)#.view(1, -1)
        output = output.to(device)
        loss = F.nll_loss(F.log_softmax(output, dim=1), self.old_y)
        loss.backward()

        for n, p in self.model.named_parameters():
            #print(f"p grad data shape = {p.grad.data.shape}")
            precision_matrices[n].data += p.grad.data ** 2 / len(self.old_x) #то, чточ делим на число это типа матожидание берем

        precision_matrices = {n: p for n, p in precision_matrices.items()}
        return precision_matrices

    def penalty(self, model: nn.Module):
        loss = 0
        for n, p in model.named_parameters():
            _loss = self._precision_matrices[n].data * (p - self._old_params[n].data) ** 2
            loss += _loss.sum()
        return loss


In [ ]:
#определение обычной функции обучения
def normal_train(X_trains, y_trains, X_tests, y_tests, model = yann_lecun, optimizer = my_optimizer, epochs = 5, batch_sizes = 100):
    model.train() #выставляем, что она меняет свои коэф
    batch_size = batch_sizes
    test_accuracy_history = []
    test_loss_history = []

    X_tests = X_tests.to(device)
    y_tests = y_tests.to(device)

    for epoch in range(epochs):
        order = np.random.permutation(len(X_trains))
        for start_index in range(0,len(X_trains), batch_size):
            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_trains[batch_indexes].to(device)
            y_batch = y_trains[batch_indexes].to(device)

            X_batch, y_batch = variable(X_batch), variable(y_batch)
            optimizer.zero_grad()
            preds = model.forward(X_batch)
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            optimizer.step()
        
        test_preds = model.forward(X_tests)
        test_loss_history.append(loss(test_preds, y_tests).data.cpu())

        accuracy = (test_preds.argmax(dim = 1) == y_tests).float().mean().data.cpu()
        test_accuracy_history.append(accuracy)
        print(accuracy)  

#определение функции обучения с EWC

def ewc_train(X_trains, y_trains, X_tests, y_tests, old_x_test, old_y_test, 
              ewc: EWC, importance, number, epochs = 7, model = yann_lecun, optimizer = my_optimizer, batch_sizes = 100):
    model.train() #выставляем, что она меняет свои коэф
    #epoch_loss = 0
    batch_size = batch_sizes
    test_accuracy_history = []
    test_loss_history = []

    X_tests = X_tests.to(device)
    y_tests = y_tests.to(device)

    for epoch in range(epochs):
        order = np.random.permutation(len(X_trains))
        for start_index in range(0,len(X_trains), batch_size):
            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_trains[batch_indexes].to(device)
            y_batch = y_trains[batch_indexes].to(device)

            X_batch, y_batch = variable(X_batch), variable(y_batch)
            optimizer.zero_grad()
            preds = model.forward(X_batch)
            loss_value = loss(preds, y_batch) + importance * ewc.penalty(model)
            loss_value.backward()
            optimizer.step()
        
        test_preds = model.forward(X_tests)
        test_loss_history.append(loss(test_preds, y_tests).data.cpu())

        accuracy = (test_preds.argmax(dim = 1) == y_tests).float().mean().data.cpu()
        print('current accuracy: ', accuracy)
        test_accuracy_history.append(accuracy)
        old_accuracy = test_accuracy(old_x_test, old_y_test)
        print('old accuracy: ', old_accuracy)
        if number == 1:
            #if accuracy >= 0.2:
            #    #importance *= 2
            #    #print("importance = ", importance)
            print('accuracy on 01: ', test_accuracy(X_test_01,y_test_01),'\n')
        elif number == 2:
            print('accuracy on 01: ', test_accuracy(X_test_01,y_test_01))
            print('accuracy on 23: ', test_accuracy(X_test_23,y_test_23),'\n')
        elif number == 3:
            print('accuracy on 01: ', test_accuracy(X_test_01,y_test_01))
            print('accuracy on 23: ', test_accuracy(X_test_23,y_test_23))
            print('accuracy on 45: ', test_accuracy(X_test_45,y_test_45),'\n')
        elif number == 4:
            print('accuracy on 01: ', test_accuracy(X_test_01,y_test_01))
            print('accuracy on 23: ', test_accuracy(X_test_23,y_test_23))
            print('accuracy on 45: ', test_accuracy(X_test_45,y_test_45))
            print('accuracy on 67: ', test_accuracy(X_test_67,y_test_67),'\n')

        
        


In [ ]:
pytorch_total_params = sum(p.numel() for p in yann_lecun.parameters())
print(pytorch_total_params)

62768


In [ ]:
normal_train(X_train_01,y_train_01,X_test_01,y_test_01)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9995)
tensor(0.9995)
tensor(1.)
tensor(0.9991)
tensor(1.)


In [ ]:
old_tasks = X_train_01
old_y = y_train_01
old_x_test = X_test_01
old_y_test = y_test_01
old_y = old_y.to(device)
my_ewc = EWC(old_tasks, old_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
ewc_train(X_train_23,y_train_23,X_test_23,y_test_23,old_x_test, old_y_test, my_ewc, number = 1, epochs = 7, importance = 2.5e14)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


current accuracy:  tensor(0.)
old accuracy:  tensor(0.9995)
accuracy on 01:  tensor(0.9995) 

current accuracy:  tensor(0.)
old accuracy:  tensor(1.)
accuracy on 01:  tensor(0.9995) 

current accuracy:  tensor(0.)
old accuracy:  tensor(0.9995)
accuracy on 01:  tensor(1.) 

current accuracy:  tensor(0.)
old accuracy:  tensor(1.)
accuracy on 01:  tensor(0.9995) 

current accuracy:  tensor(0.)
old accuracy:  tensor(0.9995)
accuracy on 01:  tensor(1.) 

current accuracy:  tensor(0.)
old accuracy:  tensor(1.)
accuracy on 01:  tensor(0.9991) 

current accuracy:  tensor(0.8555)
old accuracy:  tensor(0.1683)
accuracy on 01:  tensor(0.1612) 



In [ ]:
cur_x_test = X_test_23
cur_x_test = cur_x_test.to(device)
cur_y_test = y_test_23

test_accuracy(cur_x_test, cur_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.8546)

In [ ]:
old_tasks = torch.cat((old_tasks,X_train_23))
old_x_test = torch.cat((old_x_test, cur_x_test))
old_y_test = torch.cat((old_y_test,cur_y_test))
y_train_23 = y_train_23.to(device)
old_y = torch.cat((old_y, y_train_23))
old_y = old_y.to(device)
my_ewc = EWC(old_tasks, old_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
ewc_train(X_train_45,y_train_45,X_test_45,y_test_45, old_x_test, old_y_test, my_ewc, number = 2, epochs = 7, importance = 6e11)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


current accuracy:  tensor(0.)
old accuracy:  tensor(0.4895)
accuracy on 01:  tensor(0.1735)
accuracy on 23:  tensor(0.8604) 

current accuracy:  tensor(0.0016)
old accuracy:  tensor(0.4960)
accuracy on 01:  tensor(0.1839)
accuracy on 23:  tensor(0.8585) 

current accuracy:  tensor(0.0443)
old accuracy:  tensor(0.4799)
accuracy on 01:  tensor(0.1593)
accuracy on 23:  tensor(0.8477) 

current accuracy:  tensor(0.1478)
old accuracy:  tensor(0.4638)
accuracy on 01:  tensor(0.1660)
accuracy on 23:  tensor(0.8110) 

current accuracy:  tensor(0.2236)
old accuracy:  tensor(0.4323)
accuracy on 01:  tensor(0.1560)
accuracy on 23:  tensor(0.7742) 

current accuracy:  tensor(0.2791)
old accuracy:  tensor(0.4094)
accuracy on 01:  tensor(0.1437)
accuracy on 23:  tensor(0.7179) 

current accuracy:  tensor(0.3106)
old accuracy:  tensor(0.3784)
accuracy on 01:  tensor(0.1314)
accuracy on 23:  tensor(0.6675) 



In [ ]:
cur_x_test = X_test_45
cur_y_test = y_test_45

test_accuracy(cur_x_test, cur_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.3250)

In [ ]:
old_x_test = torch.cat((old_x_test,cur_x_test.to(device)))
old_y_test = torch.cat((old_y_test,cur_y_test))
test_accuracy(old_x_test, old_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.3595)

In [ ]:
old_tasks = torch.cat((old_tasks,X_train_45))
y_train_45 = y_train_45.to(device)
old_y = torch.cat((old_y, y_train_45))
old_y = old_y.to(device)
my_ewc = EWC(old_tasks, old_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
ewc_train(X_train_67,y_train_67,X_test_67,y_test_67, old_x_test, old_y_test, my_ewc, number = 3, epochs = 7, importance = 4e11)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


current accuracy:  tensor(0.0337)
old accuracy:  tensor(0.3635)
accuracy on 01:  tensor(0.1494)
accuracy on 23:  tensor(0.6322)
accuracy on 45:  tensor(0.3191) 

current accuracy:  tensor(0.0932)
old accuracy:  tensor(0.3537)
accuracy on 01:  tensor(0.1655)
accuracy on 23:  tensor(0.6185)
accuracy on 45:  tensor(0.2978) 

current accuracy:  tensor(0.1667)
old accuracy:  tensor(0.3276)
accuracy on 01:  tensor(0.1740)
accuracy on 23:  tensor(0.5642)
accuracy on 45:  tensor(0.2663) 

current accuracy:  tensor(0.2628)
old accuracy:  tensor(0.3238)
accuracy on 01:  tensor(0.2109)
accuracy on 23:  tensor(0.5485)
accuracy on 45:  tensor(0.2391) 

current accuracy:  tensor(0.3434)
old accuracy:  tensor(0.3114)
accuracy on 01:  tensor(0.2255)
accuracy on 23:  tensor(0.5122)
accuracy on 45:  tensor(0.2006) 

current accuracy:  tensor(0.4245)
old accuracy:  tensor(0.2917)
accuracy on 01:  tensor(0.2222)
accuracy on 23:  tensor(0.5064)
accuracy on 45:  tensor(0.1601) 

current accuracy:  tensor(0.

In [ ]:
cur_x_test = X_test_67
cur_y_test = y_test_67

test_accuracy(cur_x_test, cur_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.4602)

In [ ]:
old_x_test = torch.cat((old_x_test,cur_x_test.to(device)))
old_y_test = torch.cat((old_y_test,cur_y_test))
test_accuracy(old_x_test, old_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.3189)

In [ ]:
old_tasks = torch.cat((old_tasks,X_train_67))


y_train_67 = y_train_67.to(device)
old_y = torch.cat((old_y, y_train_67))
old_y = old_y.to(device)
my_ewc = EWC(old_tasks, old_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
ewc_train(X_train_89,y_train_89,X_test_89,y_test_89,old_x_test, old_y_test, my_ewc, number = 4, epochs = 7, importance = 4e11)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


current accuracy:  tensor(0.1372)
old accuracy:  tensor(0.3111)
accuracy on 01:  tensor(0.2563)
accuracy on 23:  tensor(0.4696)
accuracy on 45:  tensor(0.0859)
accuracy on 67:  tensor(0.4245) 

current accuracy:  tensor(0.2743)
old accuracy:  tensor(0.2905)
accuracy on 01:  tensor(0.2534)
accuracy on 23:  tensor(0.4682)
accuracy on 45:  tensor(0.0811)
accuracy on 67:  tensor(0.3686) 

current accuracy:  tensor(0.3424)
old accuracy:  tensor(0.2669)
accuracy on 01:  tensor(0.2520)
accuracy on 23:  tensor(0.4682)
accuracy on 45:  tensor(0.0496)
accuracy on 67:  tensor(0.3127) 

current accuracy:  tensor(0.4049)
old accuracy:  tensor(0.2552)
accuracy on 01:  tensor(0.2582)
accuracy on 23:  tensor(0.4305)
accuracy on 45:  tensor(0.0502)
accuracy on 67:  tensor(0.2729) 

current accuracy:  tensor(0.4407)
old accuracy:  tensor(0.2415)
accuracy on 01:  tensor(0.2421)
accuracy on 23:  tensor(0.4354)
accuracy on 45:  tensor(0.0438)
accuracy on 67:  tensor(0.2719) 

current accuracy:  tensor(0.46

In [ ]:
cur_x_test = X_test_89
cur_y_test = y_test_89

test_accuracy(cur_x_test, cur_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.4871)

In [ ]:
old_x_test = torch.cat((old_x_test,cur_x_test.to(device)))
old_y_test = torch.cat((old_y_test,cur_y_test))
test_accuracy(old_x_test, old_y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.2875)